In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import utide
import datetime as d
# import sklearn.gaussian_process.kernels as ke

In [2]:
def read_data(base_path):
    name = ["t1", "t2", "t3", "t4", "v1", "v2", "v3", "v4"]
    data = []
    for i in name:
        path = base_path + i + ".mat"
        tmp = loadmat(path)
        tmp_data = tmp[i]
        
        data.append(tmp_data.reshape(-1))
    
    return data

In [3]:
def get_locations(node_number):
    f = open("./Data/fort.14")
    f.readline()
    f.readline()

    all_data = []
    for i in range(5200):
        read_txt = f.readline().split()
        for i in range(4):
            read_txt[i] = float(read_txt[i])

        all_data.append(read_txt)
    
    # returen the longitutde and latitude corresponds to the node
    return all_data[node_number][1], all_data[node_number][2]

In [4]:
def node(base_path, node_number):
    long_node, lat_node = get_locations(node_number)
    
    data = read_data(base_path)
    
    u_node = np.array([])
    v_node = np.array([])
    for i in range(4):
        u_node = np.concatenate((u_node, data[4+i][192:].real))
        v_node = np.concatenate((v_node, data[4+i][192:].imag))
    
    for i in range(4):
        data[i] = data[i] / 86400
        
    # 直接计算四个日期间用小数表示的形式直接进行计算！！！后期整合再全部使用datetime
    
    t1 = (data[0][192:])
    t2 = (data[1][192:] + (np.datetime64('2014-02-28') - np.datetime64('2013-12-30')).astype(int))
    t3 = (data[2][192:] + (np.datetime64('2014-04-29') - np.datetime64('2013-12-30')).astype(int))
    t4 = (data[3][192:] + (np.datetime64('2014-06-28') - np.datetime64('2013-12-30')).astype(int))
    
    
    
    t_raw_node = np.concatenate((t1,t2,t3,t4))
    u_raw_node = u_node
    v_raw_node = v_node
    
    return t_raw_node, u_raw_node, v_raw_node, long_node, lat_node

# t,u,v,lo,la = node('./Data/1/', 4043)

# Computes the constituent frquencies for the time length of the data and sampling frequency 

In [5]:
node_list = [4034, 3832, 3719, 3721, 3839, 3947, 4048, 4045, 3942, 3939, 4038, 3833, 3836, 3944, 3941]
t_raw_node, u_raw_node, v_raw_node, long_node, lat_node = [], [], [], [], []
for i in range(15):
    path = './Data/' + str(i+1) + '/'
    t,u,v,lo,la = node(path, node_list[i])
    t_raw_node.append(t)
    u_raw_node.append(u)
    v_raw_node.append(v)
    long_node.append(lo)
    lat_node.append(la)


t_raw_node_reduced, u_raw_node_reduced, v_raw_node_reduced = t_raw_node[0][::4], u_raw_node[0][::4], v_raw_node[0][::4]
for i in range(14):
    t_raw_node_reduced = np.vstack((t_raw_node_reduced, t_raw_node[i+1][::4]))
    u_raw_node_reduced = np.vstack((u_raw_node_reduced, u_raw_node[i+1][::4]))
    v_raw_node_reduced = np.vstack((v_raw_node_reduced, v_raw_node[i+1][::4]))

In [6]:
tr_start = 0
tr_end = 2184
test_start = 2185
test_end = 2184*2 + 1

t_raw_utide, u_raw_utide, v_raw_utide = t_raw_node_reduced[4, tr_start:tr_end], u_raw_node_reduced[4, tr_start:tr_end], v_raw_node_reduced[4, tr_start:tr_end]

lat = lat_node[0]

coef = utide.solve(t_raw_utide, u_raw_utide, v_raw_utide, lat=lat, method='ols')
A1 = coef.name

num_const = len(A1)

ut = loadmat("./UTide/ut_constants.mat")

A = ut['const'][0][0][0]
freq = ut["const"][0][0][1]

selected_const_index = []  # location/index of the selected constituent in the list of all constituents

for j in A1:
    index = 0
    length = len(j)
    for k in A:
        if j == k[:length]:
            selected_const_index.append(index)
            break
        index = index + 1

selected_const_freq = freq[selected_const_index].reshape(-1) # frequency of the selected constituents

solve: matrix prep ... solution ... diagnostics ... done.


# Convert  latitude and longitude of the locations to spatial distances from a mean location

In [7]:
lat_node = np.array(lat_node)
long_node = np.array(long_node)

mean_lat, mean_long = np.mean(lat_node), np.mean(long_node)
lat0, long0 = np.min(lat_node), np.min(long_node)

num_location = 15
dlong, dlat = [], []
for i in range(num_location):
    theta1 = lat_node[i]
    theta2 = lat_node[i]
    lambda1 = mean_long
    lambda2 = long_node[i]
    
    tmp_long = 6371 * np.arccos( np.sin( theta1 / 180 * np.pi ) * np.sin( theta2 / 180 * np.pi ) + np.cos( theta1 / 180 * np.pi) * np.cos( theta2 / 180 * np.pi) * np.cos( ( lambda2 - lambda1 ) / 180 * np.pi) )
    
    theta1 = mean_lat
    theta2 = lat_node[i]
    lambda1 = long_node[i]
    lambda2 = long_node[i]
    
    tmp_lat = 6371 * np.arccos( np.sin( theta1 / 180 * np.pi ) * np.sin( theta2 / 180 * np.pi ) + np.cos( theta1 / 180 * np.pi) * np.cos( theta2 / 180 * np.pi) * np.cos( ( lambda2 - lambda1 ) / 180 * np.pi) )
    
    dlong.append(tmp_long)
    dlat.append(tmp_lat)

dlong = np.array(dlong)
dlat = np.array(dlat)

for i in range(num_location):
    if long_node[i] < mean_long:
        dlong[i] = - dlong[i]
    if lat_node[i] < mean_lat:
        dlat[i] = - dlat[i]

# Prepare the inputs x and outputs y for training the Gaussian Process

In [8]:
num_pts=int(168/(7*24))
combn = loadmat("./sampling/combn_hour.mat")["combn"]

In [9]:
x_t1 = np.diag(t_raw_node_reduced[combn[:,0]-1])
x_t2 = np.diag(t_raw_node_reduced[combn[:,1]-1])

x_dlat1 = dlat[combn[:,0]-1]
x_dlat2 = dlat[combn[:,1]-1]

x_dlong1 = dlong[combn[:,0]-1]
x_dlong2 = dlong[combn[:,1]-1]

num = len(x_t1)+len(x_t2)
x_t = np.concatenate((x_t1, x_t2)).reshape(num, 1)
x_dlat = np.concatenate((x_dlat1, x_dlat2)).reshape(num, 1)
x_dlong = np.concatenate((x_dlong1, x_dlong2)).reshape(num, 1)

x1 = np.hstack((x_t, x_dlat, x_dlong))

y_u1 = np.diag(u_raw_node_reduced[combn[:,0]-1])
y_u2 = np.diag(u_raw_node_reduced[combn[:,1]-1])

y1a = np.concatenate((y_u1, y_u1)).reshape(num,1)

y_v1 = np.diag(v_raw_node_reduced[combn[:,0]-1])
y_v2 = np.diag(v_raw_node_reduced[combn[:,1]-1])

y1b = np.concatenate((y_v1, y_v1)).reshape(num,1)

t_specific = np.vstack((x_t1, x_t2)).T
u_specific = np.vstack((y_u1, y_u2)).T
v_specific = np.vstack((y_v1, y_v2)).T

y1 = np.hstack((y1a, y1b))